In [ ]:
!wget https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip
!unzip sms+spam+collection.zip

--2024-12-10 20:22:50--  https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘sms+spam+collection.zip’

sms+spam+collection     [  <=>               ] 198.65K   771KB/s    in 0.3s    

2024-12-10 20:22:51 (771 KB/s) - ‘sms+spam+collection.zip’ saved [203415]

Archive:  sms+spam+collection.zip
  inflating: SMSSpamCollection       
  inflating: readme                  


In [ ]:
#regular Expression ->  특정한 규칙을 가진 문자열의 패턴을 정의하고, 이를 활용하여 텍스트를 검색, 매칭, 변환하는 데 사용하는 강력한 도구
#데이터 전처리
import re
import random

data = []

with open("SMSSpamCollection", "r") as f:
  for line in f:
    label, txt = line.split("\t")
    bow = set(re.findall("[a-z0-9_]+", txt.lower()))
    data.append((label, bow))


#data 섞기
random.shuffle(data)

n_train = int(len(data) * 0.8)

#데이터 spilt
train = data[:n_train]
test = data[n_train:]


In [7]:


data = []
data = [
    ['ham', ["회의", "자료", "첨부"]],
  ['ham', ["회의", "파일", "확인"]],
    ['ham', ["자료", "확인", "첨부"]],
   ['ham', ["회의", "파일", "자료"]],
    ['ham', ["확인", "파일", "첨부"]],
  ['ham', ["회의", "확인", "자료"]],
['spam', ["무료", "대출", "혜택"]],
['spam', ["무료", "확인", "대출"]],
['spam', ["자료", "무료", "첨부"]],
['spam', ["대출", "혜택", "확인"]]
]
"""
for line in data_1:
  label, txt = line.split("\t")
  bow = set(txt.lower())
  data.append((label, bow))

print(data)
"""

'\nfor line in data_1:\n  label, txt = line.split("\t")\n  bow = set(txt.lower())\n  data.append((label, bow))\n\nprint(data)\n'

In [11]:
#라플라스 스무딩 -> a개 만큼 모든 단어를 포함한 일반/스팸 메일이 있다고 가정
import math


alpha = 1

data_len = len(data)
print(data_len)
n_ham = sum(1 for label, bow in data if label == 'ham')
n_spam = sum(1 for label, bow in data if label == 'spam')

prior_spam = math.log((n_spam+alpha) / (data_len + 2*alpha))
prior_ham = math.log((n_ham+alpha) / (data_len + 2*alpha))

print(prior_spam, prior_ham)

10
-0.8754687373538999 -0.5389965007326869


In [13]:
from collections import defaultdict

#빈도 계산
words_ham = defaultdict(lambda : 0)
words_spam = defaultdict(lambda : 0)

#bow는 word의 집
for label, bow in data:
  for word in bow:
    if label == 'spam':
      words_spam[word] += 1
    else:
      words_ham[word] += 1

# 각 단어의 가능도 계산
like_ham = defaultdict(lambda : math.log(alpha/ (n_ham + alpha)))
like_spam = defaultdict(lambda : math.log(alpha/ (n_spam + alpha)))


#word는 단어 cnt는 빈도수

for word , cnt in words_ham.items():
  like_ham[word] = math.log((cnt + alpha) / (n_ham + alpha))

for word , cnt in words_spam.items():
  like_spam[word] = math.log((cnt + alpha) / (n_spam + alpha))

sorted(like_spam.items(), key=lambda x : x[1], reverse=True)[:20]

#print(words_spam)
#빈도 수 확인
#sorted(words_spam.items(), key=lambda x : x[1], reverse=True)[:20]

[('무료', -0.2231435513142097),
 ('대출', -0.2231435513142097),
 ('혜택', -0.5108256237659907),
 ('확인', -0.5108256237659907),
 ('자료', -0.916290731874155),
 ('첨부', -0.916290731874155)]

In [ ]:
print(like_ham)

defaultdict(<function <lambda> at 0x78fc28f62830>, {'buy': -4.388255126973271, 'hey': -3.7376685572073542, 'want': -3.4074271760430133, 'to': -1.352703342463677, 'anything': -4.2521232177996895, 'you': -1.290607748797005, 'do': -2.726068337227966, 'i': -0.8625095572792351, 'day': -3.2355770424715287, 'loving': -5.774543238125714, 'everything': -5.214931021595478, 'babe': -4.34743321578618, 'goes': -5.1684112224097944, 'love': -3.3541827020334294, 'are': -2.5999756570207095, 'hope': -3.816805788252113, 'miss': -4.388255126973271, 'well': -3.684746211781639, 'that': -2.3010332697846443, 'how': -2.84335737456356, 'already': -3.9553917766577174, 'what': -2.9661528939753965, 'up': -2.847811722928814, 'my': -2.0919415206804004, 'kiss': -5.315013978901841, 'yep': -6.062222532826723, 'oh': -3.89000910292332, 'ok': -2.879560406842479, 'didnt': -5.123959657466541, 'know': -3.0665008147667137, 'baby': -5.1684112224097944, 'am': -3.1595911836322155, 'jontin': -8.259358226211992, 'meant': -5.861553

In [17]:
#score 계산
def predict(bow):
  ham_score = prior_ham
  spam_score = prior_spam

  for word in bow:
    ham_score += like_ham[word]
    spam_score += like_spam[word]


  print(ham_score)
  print(spam_score)

  if spam_score > ham_score:
    return 'spam'
  else:
    return 'ham'

In [19]:
a = predict(["자료", "첨부", "파일", "확인"])
b = predict(["무료", "대출", "확인", "자료"])

-2.331172549845958
-4.828313737302301
-5.103761272085739
-2.748872195622465


In [ ]:
fn, fp, tn, tp = 0, 0, 0, 0
#
#value_set = set(my_dict.values())
data = [
    ('spam', {'inclusive', 'comuk', 'unsubscribe', 'pls', 'www'}),
    ('ham', {'buy', 'hey', 'want', 'to', 'anything', 'you', 'do'})
]

result = predict(['ringtone', 'not', 'will', 'you', 'reply', 'if', 'please', 'no', 'be', 'mobile', 'uk', 'your', 'confirm', 'replying', 'month', 'yes', 'by', 'for', 'to', 'charged', 'subscription', 'thanks', 'or', '5'])

print(result)


spam


In [ ]:
#test 해보기 -> 어떤게 True인 것에 따라 달라진다.
# TP 양성인데 양성 판정
# TN 양성인데 음성 판정
# FP 음성인데 양성 판정
# FN 음성인데 음성 판정

fn, fp, tn, tp = 0, 0, 0, 0

for label, bow in test:
  result = predict(bow)
  if result == 'spam' and label == 'spam':
    tp += 1
  elif result == 'spam' and label == 'ham':
    fp += 1
  elif result == 'ham' and label == 'spam':
    fn += 1
  elif result == 'ham' and label == 'ham':
    tn += 1

print(tp, tn, fp, fn)

144 928 35 8


In [1]:
tp = 20
tn = 7
fp = 3
fn = 80

In [2]:
acc = (tp + tn) / (tp + tn + fp + fn)
print(acc)

0.24545454545454545


In [ ]:
#F1 score 2* recall * prec / (recall + prec)
# recall -> 전체 중에 맞추었는데 몇개를 검출했냐
# precsion -> 예측한 것에서 실제 맞추는 것
prec = tp / (tp + fp)
recall = tp / (tp + fn)

f1 = 2 * prec * recall / (prec + recall)
print(f1)

0.8700906344410877
